In [6]:
#First we import the requested modules
import pandas as pd #Module for the dataframes
import geopandas as gpd #For the geodataframes
import requests 
import json
from shapely.geometry import Point, LineString
from shapely.geometry import shape
from shapely import wkt
import fiona
import asyncio
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

#Function to ensure we get a good response for the request
def requests_retry_session(retries=3,backoff_factor=0.3,status_forcelist=(500, 502, 504),session=None):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_access_token(email,password) :
    '''
    Returns the access token of the EMT Madrid API
        
        Parameters
        ----------
        email : string
            The email of the account
        password : string
            Password of the account
    '''
    response = requests_retry_session().get(
        'https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/',
        headers={
            'email':email,
            'password':password
        },
        timeout=5
    )
    json_response = response.json()
    accessToken = json_response['data'][0]['accessToken']
    return accessToken

def get_stops_of_line(lineId,direction,accessToken) :
    """
    Returns the list of stops for the line and direction desired

        Parameters
        ----------
        lineId : string
            The line id
        direction : string
            The direction (1 or 2)
        accessToken: string
            The accessToken obtained in the login
    """
    
    response = requests_retry_session().get(
        'https://openapi.emtmadrid.es/v2/transport/busemtmad/lines/{}/stops/{}/'.format(lineId,direction),
        headers = {'accessToken': accessToken},
        timeout = 5
    )
    
    #We turn the data of the stops from the response into a dataframe
    stops_data = pd.DataFrame(response.json()['data'][0]['stops'])   
    #And transform the geometry coordinates into point objects
    stops_data['geometry'] = [shape(i) for i in stops_data['geometry']]
    return stops_data

In [7]:
import datetime
#Check if time is inside range
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

now = datetime.datetime.now()
now.time()

datetime.time(12, 58, 40, 765277)

In [8]:
start_time = datetime.time(6,0,0)
end_time = datetime.time(20,50,0)
time_in_range(start_time,end_time,now.time())

True

In [10]:
import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

haversine((1,1),(2,2))

157269.85297319587

In [4]:
#Our access credentials
email = 'a.jarabo@alumnos.upm.es'
password = 'YXKE8fDrX.pMZ58'
accessToken = get_access_token(email,password)
accessToken

'98a979c4-0886-460f-933f-10df1abccf40'

In [17]:
lineId = '92' #Línea G
direction = '1'
stops_G_dir1 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_G_dir1

['2419',
 '3688',
 '1684',
 '1686',
 '1687',
 '1689',
 '1691',
 '1693',
 '4285',
 '5370',
 '1696',
 '1694',
 '4287',
 '4289',
 '4290']

In [18]:
lineId = '92' #Línea G
direction = '2'
stops_G_dir2 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_G_dir2

['4290', '4288', '1692', '1690', '1688', '1685', '1331', '2419']

In [19]:
lineId = '91' #Línea F
direction = '1'
stops_F_dir1 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_F_dir1 

['2480',
 '1418',
 '1416',
 '2712',
 '3278',
 '5140',
 '3292',
 '3274',
 '1693',
 '4285',
 '5370',
 '1696',
 '1694',
 '4287',
 '4289',
 '4290']

In [20]:
lineId = '91' #Línea F
direction = '2'
stops_F_dir2 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_F_dir2 

['4290',
 '4288',
 '4291',
 '3273',
 '3276',
 '5316',
 '185',
 '1415',
 '1417',
 '4284',
 '2480']

In [45]:
stops = stops_F_dir1 + stops_F_dir2 + stops_G_dir1 + stops_G_dir2
unique_stops = list(set(stops))
unique_stops

['4287',
 '3278',
 '1692',
 '2712',
 '2480',
 '185',
 '1331',
 '1416',
 '1415',
 '2419',
 '5140',
 '5370',
 '1417',
 '4289',
 '1690',
 '4288',
 '1696',
 '1694',
 '3273',
 '1686',
 '1693',
 '5316',
 '1691',
 '1688',
 '1687',
 '3276',
 '4285',
 '1689',
 '1418',
 '3292',
 '1685',
 '4284',
 '4290',
 '3688',
 '4291',
 '1684',
 '3274']

In [32]:
lineId = '068' #Línea C1
direction = '1'
stops_C1_dir1 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_C1_dir1

['1890',
 '1890',
 '2693',
 '2695',
 '2696',
 '421',
 '420',
 '4678',
 '2698',
 '4676',
 '2700',
 '789',
 '786',
 '811',
 '3829',
 '1427',
 '148',
 '1252',
 '2152',
 '2154',
 '2156',
 '814',
 '816',
 '2213',
 '2211',
 '2049',
 '2047',
 '2045',
 '2178',
 '83',
 '84',
 '320',
 '2355']

In [33]:
lineId = '068' #Línea C1
direction = '2'
stops_C1_dir2 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_C1_dir2

['2355',
 '2357',
 '2359',
 '2361',
 '2363',
 '2365',
 '609',
 '1562',
 '602',
 '600',
 '1750',
 '172',
 '734',
 '736',
 '738',
 '4816',
 '4021',
 '743',
 '2704',
 '2706',
 '2708',
 '2710',
 '185',
 '1415',
 '1417',
 '1890',
 '1890']

In [36]:
lineId = '069' #Línea C2
direction = '1'
stops_C2_dir1 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_C2_dir1

['1940',
 '1418',
 '1416',
 '2712',
 '2713',
 '5141',
 '2709',
 '2707',
 '2705',
 '4514',
 '4022',
 '3687',
 '737',
 '735',
 '193',
 '173',
 '854',
 '599',
 '601',
 '603',
 '4070',
 '606',
 '2367',
 '2366',
 '2364',
 '2362',
 '2360',
 '2358',
 '2356',
 '5104']

In [37]:
lineId = '069' #Línea C2
direction = '2'
stops_C2_dir2 = get_stops_of_line(lineId,direction,accessToken)['stop'].tolist()
stops_C2_dir2

['5104',
 '85',
 '4985',
 '1401',
 '1401',
 '2046',
 '2048',
 '2210',
 '2212',
 '817',
 '2214',
 '2157',
 '2155',
 '2703',
 '2702',
 '149',
 '1428',
 '782',
 '784',
 '5296',
 '788',
 '2701',
 '4677',
 '4679',
 '5394',
 '419',
 '422',
 '2697',
 '2694',
 '5449',
 '1940']

In [48]:
stops_circular= stops_C1_dir1 + stops_C1_dir2 + stops_C2_dir1 + stops_C2_dir2
unique_stops_circular = list(set(stops_circular))
unique_stops_circular

['2701',
 '2211',
 '814',
 '5296',
 '734',
 '172',
 '735',
 '1415',
 '4021',
 '5141',
 '2045',
 '422',
 '4514',
 '603',
 '2152',
 '738',
 '2702',
 '2706',
 '1418',
 '817',
 '2154',
 '816',
 '2366',
 '2709',
 '4985',
 '1940',
 '2362',
 '2363',
 '4678',
 '2700',
 '419',
 '1890',
 '2156',
 '601',
 '2712',
 '185',
 '609',
 '788',
 '2357',
 '1401',
 '84',
 '421',
 '2157',
 '2365',
 '782',
 '1428',
 '2697',
 '2704',
 '1562',
 '602',
 '2356',
 '854',
 '606',
 '2359',
 '2696',
 '2361',
 '4816',
 '149',
 '786',
 '85',
 '320',
 '2210',
 '193',
 '737',
 '2698',
 '2360',
 '743',
 '1417',
 '4070',
 '2358',
 '2355',
 '2713',
 '3829',
 '2367',
 '2178',
 '736',
 '2710',
 '2364',
 '5394',
 '2695',
 '599',
 '2046',
 '5449',
 '1427',
 '784',
 '4677',
 '148',
 '2708',
 '83',
 '1416',
 '4676',
 '600',
 '1750',
 '4679',
 '420',
 '2213',
 '2048',
 '789',
 '2155',
 '3687',
 '2049',
 '2694',
 '4022',
 '2693',
 '2705',
 '2047',
 '2703',
 '5104',
 '2214',
 '173',
 '1252',
 '811',
 '2212',
 '2707']

In [49]:
len(set(unique_stops + unique_stops_circular))

145

In [50]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/globe_contours.csv')
df.head()

,lat-1,lon-1,lat-2,lon-2,lat-3,lon-3,lat-4,lon-4,lat-5,lon-5,...,lat-14,lon-14,lat-15,lon-15,lat-16,lon-16,lat-17,lon-17,lat-18,lon-18
0,28.888688,0.0,35.663535,0.0,42.170841,0.0,47.912021,0.0,52.500000,0.0,...,-52.500000,0.0,-55.736177,0.0,-57.642093,0.0,-58.437121,0.0,-58.470238,0.0
1,28.680866,2.5,35.404502,2.5,41.868417,2.5,47.580464,2.5,52.158174,2.5,...,-52.158174,2.5,-55.404620,2.5,-57.339669,2.5,-58.178089,2.5,-58.262416,2.5
2,28.063715,5.0,34.635274,5.0,40.970334,5.0,46.595867,5.0,51.143084,5.0,...,-51.143084,5.0,-54.420023,5.0,-56.441586,5.0,-57.408861,5.0,-57.645266,5.0
3,27.055987,7.5,33.379224,7.5,39.503880,7.5,44.988147,7.5,49.485572,7.5,...,-49.485572,7.5,-52.812302,7.5,-54.975132,7.5,-56.152811,7.5,-56.637538,7.5
4,25.688302,10.0,31.674516,10.0,37.513612,10.0,42.806152,10.0,47.236000,10.0,...,-47.236000,10.0,-50.630308,10.0,-52.984864,10.0,-54.448102,10.0,-55.269852,10.0
